# Chapter 5 The Mechanics of learning
> A summary of chapter 5 part 2 of Deep learning with PyTorch

- toc: true
- badges: true
- comments: true
- categories: [jupyter]
- image: image/chart-preview

# The magic of Autograd

In [part 1](https://girijeshcse.github.io/scribble/jupyter/2121/09/14/chapter-5.html) we saw that how we can train a linear model with backpropagation. In nutshell what we computed the gradient of a composition of functions—the model and the loss—with respect to their innermost parameters (w and b) by propagating derivatives backward using the chain rule.
      
The basic requirement here is that all functions we’re dealing
with can be differentiated analytically but as we increase the depth of the model, writing the analytical expression for the derivatives is not that easy.

This is when PyTorch tensors come to the rescue, with a PyTorch component called
*autograd*.

PyTorch tensors can remember where they come from, in terms of the operations and parent tensors that originated them, and they can automatically provide the chain of derivatives of such operations with respect to their inputs. This means **PyTorch will automatically provide the gradient of that expression with respect to its input parameters.**


## Computing the gradient automatically 

Lets rewrite out code using autograd.

In [1]:
%matplotlib inline
import numpy as np
import torch
torch.set_printoptions(edgeitems=2)

In [2]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 8.0,
                    3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u

 Recalling our model and loss function

In [4]:
def model(t_u, w, b):
    return w * t_u + b

In [5]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

Let’s again initialize a parameters tensor:

In [6]:
params = torch.tensor([1.0, 0.0], requires_grad=True)

The *requires_grad=True* argument is telling PyTorch to track the entire family tree of tensors resulting from operations on params.

In [7]:
params.grad is None

True

In [8]:
loss = loss_fn(model(t_u, *params), t_c)
loss.backward()

params.grad

tensor([4517.2969,   82.6000])

At this point, the grad attribute of params contains the derivatives of the loss with respect to each element of params.


## The forward graph and backward graph of the model as computed with autograd


![](my_icons/ch55.JPG)

When we compute our loss while the parameters w and b require gradients, in addition to performing the actual computation, PyTorch creates the autograd graph with the operations (in black circles) as nodes, as shown in the top row of figure above. When we call *loss.backward()*, PyTorch traverses this graph in the reverse direction to compute the gradients, as shown by the arrows in the bottom row of the figure.




### Accumulating Grad Funtion

Calling backward will lead derivatives to accumulate at leaf nodes. So if backward was called earlier, the loss is evaluated again, backward is called again (as in any training loop), and the gradient at each leaf is accumulated (that is, summed) on top of the one computed at the previous iteration, which leads to an ***incorrect value*** for the gradient. In order to prevent this from occurring, ***we need to zero the gradient explicitly at each iteration***. We can do this easily using the in-place zero_ method:

In [9]:
if params.grad is not None:
    params.grad.zero_()

In [10]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:  # This could be done at any point in the loop prior to calling loss.backward().
            params.grad.zero_()
        
        t_p = model(t_u, *params) 
        loss = loss_fn(t_p, t_c)
        loss.backward()
        
        with torch.no_grad():  # This is a somewhat cumbersome bit of code, but as we’ll see in the next section, it’s not an issue in practice.
            params -= learning_rate * params.grad

        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return params

In [11]:
training_loop(
    n_epochs = 5000, 
    learning_rate = 1e-2, 
    params = torch.tensor([1.0, 0.0], requires_grad=True), # <1> 
    t_u = t_un, # <2> 
    t_c = t_c)

Epoch 500, Loss 7.860115
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957698
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927830
Epoch 4000, Loss 2.927679
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647


tensor([  5.3671, -17.3012], requires_grad=True)

#  Making this more better